In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_3138/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
    
    def has_minimum_requirements(self, df, sort_by="r2", boundary=0.75):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer()
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt,Architecture
0,model_10_5_5,0.970934,0.888509,0.947529,0.914159,0.005438,0.013059,0.004635,0.009095,"Hidden Size=[60, 30], regularizer=0.02, learni..."
1,model_10_5_6,0.970769,0.865830,0.938833,0.897473,0.005469,0.015716,0.005404,0.010863,"Hidden Size=[60, 30], regularizer=0.02, learni..."
2,model_10_6_4,0.970187,0.686905,0.957118,0.925498,0.005577,0.010588,0.009473,0.010063,"Hidden Size=[60, 30], regularizer=0.02, learni..."
3,model_10_5_4,0.970183,0.909435,0.955054,0.929359,0.005578,0.010608,0.003971,0.007485,"Hidden Size=[60, 30], regularizer=0.02, learni..."
4,model_10_5_7,0.970134,0.845149,0.927554,0.880944,0.005587,0.018138,0.006400,0.012614,"Hidden Size=[60, 30], regularizer=0.02, learni..."
...,...,...,...,...,...,...,...,...,...,...
335,model_10_3_2,0.895316,0.873813,0.597888,0.844489,0.019584,0.040145,0.023792,0.032450,"Hidden Size=[60, 30], regularizer=0.02, learni..."
338,model_9_5_5,0.894289,0.762160,0.944179,0.921079,0.019776,0.010809,0.018831,0.014584,"Hidden Size=[60, 30], regularizer=0.02, learni..."
340,model_9_9_4,0.893753,0.753613,0.811401,0.820594,0.019876,0.035802,0.011047,0.024152,"Hidden Size=[60, 30], regularizer=0.02, learni..."
342,model_9_5_4,0.892134,0.771438,0.947761,0.925364,0.020179,0.010387,0.017623,0.013792,"Hidden Size=[60, 30], regularizer=0.02, learni..."
